In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

# Load data
df = pd.read_csv("MRI.csv", index_col=0)
df = df.apply(pd.to_numeric, errors='coerce')

right_pre = df.iloc[0, 0:3].values.astype(float)
right_post = df.iloc[1, 0:3].values.astype(float)
left_pre = df.iloc[0, 3:6].values.astype(float)
left_post = df.iloc[1, 3:6].values.astype(float)

means = [
    np.mean(right_pre), np.mean(right_post),
    np.mean(left_pre), np.mean(left_post)
]
sems = [
    np.std(right_pre, ddof=1)/np.sqrt(len(right_pre)),
    np.std(right_post, ddof=1)/np.sqrt(len(right_post)),
    np.std(left_pre, ddof=1)/np.sqrt(len(left_pre)),
    np.std(left_post, ddof=1)/np.sqrt(len(left_post))
]

t_right, p_right = ttest_rel(right_pre, right_post)
t_left, p_left = ttest_rel(left_pre, left_post)

# Print stats table
print("\nT-test Results:")
print("{:<15} {:>10} {:>10} {:>10} {:>10} {:>10}".format(
    "Group", "Mean1", "Mean2", "SEM1", "SEM2", "p-value"))
print("{:<15} {:>10.2f} {:>10.2f} {:>10.2f} {:>10.2f} {:>10.4f}".format(
    "Right Tumor", means[0], means[1], sems[0], sems[1], p_right))
print("{:<15} {:>10.2f} {:>10.2f} {:>10.2f} {:>10.2f} {:>10.4f}".format(
    "Left Tumor", means[2], means[3], sems[2], sems[3], p_left))

# Plotting
pre_color = "#459B3D"
post_color = "#AF4D50"

bar_width = 0.35
gap_multiplier = 1.1

labels = ['Right Tumor', 'Left Tumor']
x = np.arange(len(labels))

fig, ax = plt.subplots(figsize=(8, 6))

bars1 = ax.bar(x - gap_multiplier * bar_width / 2, [means[0], means[2]], width=bar_width,
               yerr=[sems[0], sems[2]], capsize=7, label='Pre-injection',
               color=pre_color, ecolor=pre_color, error_kw=dict(lw=2))
bars2 = ax.bar(x + gap_multiplier * bar_width / 2, [means[1], means[3]], width=bar_width,
               yerr=[sems[1], sems[3]], capsize=7, label='Post-injection',
               color=post_color, ecolor=post_color, error_kw=dict(lw=2))

ax.set_ylabel('MRI Signal Intensity', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(['']*len(labels))

offset = -15
ax.annotate('Right Tumor', xy=(x[0], 0), xytext=(x[0], offset),
            ha='center', va='top', fontsize=16, annotation_clip=False)
ax.annotate('Left Tumor', xy=(x[1], 0), xytext=(x[1], offset),
            ha='center', va='top', fontsize=16, annotation_clip=False)

ax.tick_params(axis='x', length=12, width=3)
ax.tick_params(axis='y', length=12, width=3, labelsize=16)

ax.set_ylim(0, 140)
ax.set_yticks(np.arange(0, 141, 20))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)

ax.legend(fontsize=14, frameon=False, loc='upper left', bbox_to_anchor=(0.9, 1))

def p_to_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return 'n.s.'

def add_significance_line(x_start, x_end, y, star_text):
    line_height = y + 2
    star_y = line_height + 2
    ax.plot([x_start, x_start, x_end, x_end], [y, line_height, line_height, y], color='black', lw=2)
    ax.text((x_start + x_end)/2, star_y, star_text, ha='center', va='bottom', fontsize=26, color='black')

ymax_right = max(means[0], means[1]) + max(sems[0], sems[1]) + 8
ymax_left = max(means[2], means[3]) + max(sems[2], sems[3]) + 8

stars_right = p_to_stars(p_right)
if stars_right != 'n.s.':
    add_significance_line(x[0] - gap_multiplier * bar_width / 2, x[0] + gap_multiplier * bar_width / 2, ymax_right, stars_right)

stars_left = p_to_stars(p_left)
if stars_left != 'n.s.':
    add_significance_line(x[1] - gap_multiplier * bar_width / 2, x[1] + gap_multiplier * bar_width / 2, ymax_left, stars_left)

plt.tight_layout()
plt.subplots_adjust(bottom=0.22)

# Save figure as TIFF
plt.savefig("MRI signal intensity.tiff", format='tiff', dpi=600)

plt.show()
